In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import cv2, argparse
import math

# pytorch modules
import torch 
from torch.utils.data import Dataset # primitive for the data
from torch.utils.data import WeightedRandomSampler, DataLoader # wraps the data so its iterable
from torch import nn # nn class our model inherits from
import torch.optim as optim
import pandas as pd
from sklearn.metrics import confusion_matrix
import random

from PIL import Image

#### This data is from Silicon that has had a single layer of Hydrogen deposited on top. Then, some of the Hydrogen is desorbed, leaving Silicon dangling bonds. We then dose this with Arsine and Arsenic bonds to these dangling bonds. Once bonded, it has a 100% chance of incorporation after heating. 

#### After Arsenic has been introduce to the Hydrogen surface, there should be no DBs left as we try to terminate them all, but there may be some left. So, we need to include DBs in this CNN. They should look the same, so we will use some of the data already labelled that was used to train the single and double DB CNN.

In [2]:
filled_gr = np.load(r'C:\Users\nkolev\OneDrive - University College London\Documents\image processing\AsH3 identification\undosed\numpy arrays\20181015-142255_STM_AtomManipulation-Gloucester Road-Si(100)-H--9_1_0.npy')[::2,::2]
empty_gr = np.load(r'C:\Users\nkolev\OneDrive - University College London\Documents\image processing\AsH3 identification\undosed\numpy arrays\20181015-142255_STM_AtomManipulation-Gloucester Road-Si(100)-H--9_1_1_cor.npy')[::2,::2]
filled_ec = np.load(r'C:\Users\nkolev\OneDrive - University College London\Documents\image processing\AsH3 identification\undosed\numpy arrays\20181019-110413_STM_AtomManipulation-earls court-Si(100)-H--63_1_0.npy')[::-1,:][::2,::2]
empty_ec = np.load(r'C:\Users\nkolev\OneDrive - University College London\Documents\image processing\AsH3 identification\undosed\numpy arrays\20181019-110413_STM_AtomManipulation-earls court-Si(100)-H--63_1_1_cor.npy')[::-1,:][::2,::2]
filled_cl = np.load(r'C:\Users\nkolev\OneDrive - University College London\Documents\image processing\AsH3 identification\undosed\numpy arrays\20191122-195611_Chancery Lane-Si(001)H--22_2_0.npy')[::-1,:][::2,::2]
empty_cl = np.load(r'C:\Users\nkolev\OneDrive - University College London\Documents\image processing\AsH3 identification\undosed\numpy arrays\20191122-195611_Chancery Lane-Si(001)H--22_2_1_cor.npy')[::-1,:][::2,::2]
filled_k = np.load(r'C:\Users\nkolev\OneDrive - University College London\Documents\image processing\AsH3 identification\undosed\numpy arrays\20200731-132153_Kenton-Si(001)-H--12_1_0.npy')[::-1,:][::2,::2]
empty_k = np.load(r'C:\Users\nkolev\OneDrive - University College London\Documents\image processing\AsH3 identification\undosed\numpy arrays\20200731-132153_Kenton-Si(001)-H--12_1_1_cor.npy')[::-1,:][::2,::2]

filled_AsEC = np.load(r'C:\Users\nkolev\OneDrive - University College London\Documents\image processing\AsH3 identification\dosed\numpy arrays\20181026-191252_STM_AtomManipulation-Earls Court-Si(100)-H-litho-AsH3--31_1_0.npy')[::-1,:][::2,::2]
empty_AsEC = np.load(r'C:\Users\nkolev\OneDrive - University College London\Documents\image processing\AsH3 identification\dosed\numpy arrays\20181026-191252_STM_AtomManipulation-Earls Court-Si(100)-H-litho-AsH3--31_1_1_cor.npy')[::-1,:][::2,::2]
filled_AsH0 = np.load(r'C:\Users\nkolev\OneDrive - University College London\Documents\image processing\AsH3 identification\dosed\numpy arrays\20200212-183707_Hainault-Si(001)-H--54_1_0.npy')[::-1,:][::2,::2]
empty_AsH0 = np.load(r'C:\Users\nkolev\OneDrive - University College London\Documents\image processing\AsH3 identification\dosed\numpy arrays\20200212-183707_Hainault-Si(001)-H--54_1_0_cor.npy')[::-1,:][::2,::2]
filled_AsH1 = np.load(r'C:\Users\nkolev\OneDrive - University College London\Documents\image processing\AsH3 identification\dosed\numpy arrays/20200219-180436_Hainault-Si(001)-H--38_1_2.npy')[::-1,:]
empty_AsH1 =  np.load(r'C:\Users\nkolev\OneDrive - University College London\Documents\image processing\AsH3 identification\dosed\numpy arrays\20200219-180436_Hainault-Si(001)-H--38_1_3_cor.npy')[::-1,:]
filled_AsH2 = np.load(r'C:\Users\nkolev\OneDrive - University College London\Documents\image processing\AsH3 identification\dosed\numpy arrays/20200217-105833_Hainault-Si(001)-H-AsH3--30_2_2.npy')[::-1,:][::2,::2]
empty_AsH2 = np.load(r'C:\Users\nkolev\OneDrive - University College London\Documents\image processing\AsH3 identification\dosed\numpy arrays\20200217-105833_Hainault-Si(001)-H-AsH3--30_2_3_cor.npy')[::-1,:][::2,::2]

#error_gr = plt.imread(r'C:\Users\nkolev\Documents\image processing\AsH3 identification\undosed\error png\20181015-142255_STM_AtomManipulation-Gloucester Road-Si(100)-H--9_1_I.png')[::2,::2,0]
#error_ec = plt.imread(r'C:\Users\nkolev\Documents\image processing\AsH3 identification\undosed\error png\20181019-110413_STM_AtomManipulation-earls court-Si(100)-H--63_1_0I.png')[::2,::2]
#error_cl = plt.imread(r'C:\Users\nkolev\Documents\image processing\AsH3 identification\undosed\error png\20191122-195611_Chancery Lane-Si(001)H--22_2_I.png')[::2,::2]
#error_k = plt.imread(r'C:\Users\nkolev\Documents\image processing\AsH3 identification\undosed\error png\20200731-132153_Kenton-Si(001)-H--12_1_1_I.png')[::2,::2]

#error_AsEC = plt.imread(r'error png\20181026-191252_STM_AtomManipulation-Earls Court-Si(100)-H-litho-AsH3--31_1_I0.png')[::2,::2]
#error_AsH0 = plt.imread(r'error png\20200212-183707_Hainault-Si(001)-H--54_1_0I.png')
#error_AsH1 =  plt.imread(r'error png\20200219-180436_Hainault-Si(001)-H--38_1_I2.png')
#error_AsH2 = plt.imread(r'error png\20200217-105833_Hainault-Si(001)-H-AsH3--30_2_2I.png')[::2,::2]

#### labels: 1 = singling dangling bonds, 2 = double dangling bond, 3= anomaly, 4 = dimer vacancy (or any dark feature really), 5 = background Silicon/Hydrogen, 6 = step edge, 7 = As feature.


In [3]:
#load the labels from the .csv files
pd_labels_gr = pd.read_csv(r'C:\Users\nkolev\OneDrive - University College London\Documents\image processing\AsH3 identification\undosed\labels for original scans\scans_labels_pts_20181015_gloucester_rd_9_1.csv')
pd_labels_ec = pd.read_csv(r'C:\Users\nkolev\OneDrive - University College London\Documents\image processing\AsH3 identification\undosed\labels for original scans\scans_labels_pts_20181019_earls_court_63_1.csv')
pd_labels_cl = pd.read_csv(r'C:\Users\nkolev\OneDrive - University College London\Documents\image processing\AsH3 identification\undosed\labels for original scans\scans_labels_pts_20191122_chancery_lane_22_2.csv')
pd_labels_k = pd.read_csv(r'C:\Users\nkolev\OneDrive - University College London\Documents\image processing\AsH3 identification\undosed\labels for original scans\scans_labels_pts_20200731_kenton_12_1.csv')

pd_labels_AsEC = pd.read_csv(r'C:\Users\nkolev\OneDrive - University College London\Documents\image processing\AsH3 identification\dosed\labels for original scans\20181026-191252_STM_AtomManipulation-Earls Court-Si(100)-H-litho-AsH3--31_10_coords.csv')
pd_labels_AsH0 = pd.read_csv(r'C:\Users\nkolev\OneDrive - University College London\Documents\image processing\AsH3 identification\dosed\labels for original scans\20200212-183707_Hainault-Si(001)-H--54_1_0Z_coords.csv')
pd_labels_AsH1 = pd.read_csv(r'C:\Users\nkolev\OneDrive - University College London\Documents\image processing\AsH3 identification\dosed\labels for original scans\20200219-180436_Hainault-Si(001)-H--38_10_coords.csv')
pd_labels_AsH2 = pd.read_csv(r'C:\Users\nkolev\OneDrive - University College London\Documents\image processing\AsH3 identification\dosed\labels for original scans\20200217-105833_Hainault-Si(001)-H-AsH3--30_2_2_coords.csv')



#### Make a separate set of data for a validation set from scans that are not in the training data at all. This will make the validation accuracy a more reliable measure.

In [4]:
filled_cl2 = np.load(r'C:\Users\nkolev\OneDrive - University College London\Documents\image processing\AsH3 identification\undosed\numpy arrays\20191122-195611_Chancery Lane-Si(001)H--24_6_0.npy')[::-1,:][::2,::2]
empty_cl2 = np.load(r'C:\Users\nkolev\OneDrive - University College London\Documents\image processing\AsH3 identification\undosed\numpy arrays\20191122-195611_Chancery Lane-Si(001)H--24_6_1_cor.npy')[::-1,:][::2,::2]
filled_AsEC2 = np.load(r'C:\Users\nkolev\OneDrive - University College London\Documents\image processing\AsH3 identification\dosed\numpy arrays\20181123-122007_STM_AtomManipulation-Earls Court-Si(100)-H term--26_2_0.npy')[::-1,:][::2,::2]
empty_AsEC2 = np.load(r'C:\Users\nkolev\OneDrive - University College London\Documents\image processing\AsH3 identification\dosed\numpy arrays\20181123-122007_STM_AtomManipulation-Earls Court-Si(100)-H term--26_2_1_cor.npy')[::-1,:][::2,::2]

pd_labels_cl2 = pd.read_csv(r'C:\Users\nkolev\OneDrive - University College London\Documents\image processing\AsH3 identification\undosed\labels for original scans\scans_labels_pts_20191122_chancery_lane_24_6.csv')
pd_labels_AsEC2 = pd.read_csv(r'C:\Users\nkolev\OneDrive - University College London\Documents\image processing\AsH3 identification\dosed\labels for original scans\20181123-122007_STM_AtomManipulation-Earls Court-Si(100)-H term--26_2_0_coords.csv')

In [5]:
# make all images non-negative
def normalise_image(image):
    image = (image - np.min(image)) #/ (np.max(image) - np.min(image)) 
    return image

In [6]:
filled_gr = normalise_image(filled_gr)
empty_gr = normalise_image(empty_gr)
filled_ec = normalise_image(filled_ec)
empty_ec = normalise_image(empty_ec)
filled_cl = normalise_image(filled_cl)
empty_cl = normalise_image(empty_cl)
filled_k = normalise_image(filled_k)
empty_k = normalise_image(empty_k)

filled_AsEC = normalise_image(filled_AsEC)
empty_AsEC = normalise_image(empty_AsEC)
filled_AsH0 = normalise_image(filled_AsH0)
empty_AsH0 = normalise_image(empty_AsH0)
filled_AsH1 = normalise_image(filled_AsH1)
empty_AsH1 = normalise_image(empty_AsH1)
filled_AsH2 = normalise_image(filled_AsH2)
empty_AsH2 = normalise_image(empty_AsH2)
filled_cl2 = normalise_image(filled_cl2)
empty_cl2 = normalise_image(empty_cl2)
filled_AsEC2 = normalise_image(filled_AsEC2)

In [7]:
print(np.max(filled_gr), np.min(filled_gr))

0.6379094859128145 0.0


In [8]:
# array with first column being the x-coord pixel, the second being the y coord pixel and the third being the label.
coords_and_labels_array_gr = pd_labels_gr.values.copy() 
coords_and_labels_array_ec = pd_labels_ec.values.copy() 
coords_and_labels_array_cl = pd_labels_cl.values.copy() 
coords_and_labels_array_cl2 = pd_labels_cl2.values.copy() 
coords_and_labels_array_k = pd_labels_k.values.copy() 

coords_and_labels_array_AsEC = pd_labels_AsEC.values.copy() 
coords_and_labels_array_AsH0 = pd_labels_AsH0.values.copy() 
coords_and_labels_array_AsH1 = pd_labels_AsH1.values.copy() 
coords_and_labels_array_AsH2 = pd_labels_AsH2.values.copy()
coords_and_labels_array_AsEC2 = pd_labels_AsEC2.values.copy()[:-11,:3] 


In [9]:
# show the labels on the images
def show_labels_on_image(image, coords_and_labels_array):
    img_normalized = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    image_rgb = cv2.cvtColor(img_normalized, cv2.COLOR_GRAY2RGB)
    for i in range(coords_and_labels_array.shape[0]):
        x = coords_and_labels_array[i,0]
        y = coords_and_labels_array[i,1]
        label = coords_and_labels_array[i,2]
        cv2.putText(image_rgb, str(int(label)), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
    plt.figure(figsize=(20, 20))
    plt.imshow(image_rgb)
    plt.show()

In [10]:
# odd number so symmetric about middle pixel
crop_size = 14
pixel_shift = int(round(crop_size/2,0))

In [11]:
# Hainault 0 and 1 are 200nm*200nm. Need to split them up into 4 and change the coordinates labels appropriately.

In [12]:
res = 512
#filled_AsH01 = filled_AsH0[:512,:512].copy()
#filled_AsH02 = filled_AsH0[:512,512:].copy()
#filled_AsH03 = filled_AsH0[512:,:512].copy()
#filled_AsH04 = filled_AsH0[512:,512:].copy()

#error_AsH01 = error_AsH0[:512,:512].copy()
#error_AsH02 = error_AsH0[:512,512:].copy()
#error_AsH03 = error_AsH0[512:,:512].copy()
#error_AsH04 = error_AsH0[512:,512:].copy()

#coords_and_labels_array_AsH01 = coords_and_labels_array_AsH0[(coords_and_labels_array_AsH0[:,0]<513)&(coords_and_labels_array_AsH0[:,1]<513)].copy()
#coords_and_labels_array_AsH02 = coords_and_labels_array_AsH0[(coords_and_labels_array_AsH0[:,1]<513)&(coords_and_labels_array_AsH0[:,0]>512)].copy()
#coords_and_labels_array_AsH03 = coords_and_labels_array_AsH0[(coords_and_labels_array_AsH0[:,1]>512)&(coords_and_labels_array_AsH0[:,0]<513)].copy()
#coords_and_labels_array_AsH04 = coords_and_labels_array_AsH0[(coords_and_labels_array_AsH0[:,0]>512)&(coords_and_labels_array_AsH0[:,1]>512)].copy() 

# need to shift these so that the top right corner of each one is its own origin
#coords_and_labels_array_AsH02[:,0] = coords_and_labels_array_AsH02[:,0] - 513*np.ones(coords_and_labels_array_AsH02.shape[0])
#coords_and_labels_array_AsH03[:,1] = coords_and_labels_array_AsH03[:,1] - 513*np.ones(coords_and_labels_array_AsH03.shape[0])
#coords_and_labels_array_AsH04[:,:2] = coords_and_labels_array_AsH04[:,:2] - 513*np.ones( (coords_and_labels_array_AsH04.shape[0],2))

filled_AsH11 = filled_AsH1[:512,:512].copy()
filled_AsH12 = filled_AsH1[:512,512:].copy()
filled_AsH13 = filled_AsH1[512:,:512].copy()
filled_AsH14 = filled_AsH1[512:,512:].copy()

empty_AsH11 = empty_AsH1[:512,:512].copy()
empty_AsH12 = empty_AsH1[:512,512:].copy()
empty_AsH13 = empty_AsH1[512:,:512].copy()
empty_AsH14 = empty_AsH1[512:,512:].copy()

coords_and_labels_array_AsH11 = coords_and_labels_array_AsH1[(coords_and_labels_array_AsH1[:,0]<513)&(coords_and_labels_array_AsH1[:,1]<513)].copy()
coords_and_labels_array_AsH12 = coords_and_labels_array_AsH1[(coords_and_labels_array_AsH1[:,1]<513)&(coords_and_labels_array_AsH1[:,0]>512)].copy()
coords_and_labels_array_AsH13 = coords_and_labels_array_AsH1[(coords_and_labels_array_AsH1[:,1]>512)&(coords_and_labels_array_AsH1[:,0]<513)].copy()
coords_and_labels_array_AsH14 = coords_and_labels_array_AsH1[(coords_and_labels_array_AsH1[:,0]>512)&(coords_and_labels_array_AsH1[:,1]>512)].copy() 

# need to shift these so that the top right corner of each one is its own origin
coords_and_labels_array_AsH12[:,0] = coords_and_labels_array_AsH12[:,0] - 513*np.ones(coords_and_labels_array_AsH12.shape[0])
coords_and_labels_array_AsH13[:,1] = coords_and_labels_array_AsH13[:,1] - 513*np.ones(coords_and_labels_array_AsH13.shape[0])
coords_and_labels_array_AsH14[:,:2] = coords_and_labels_array_AsH14[:,:2] - 513*np.ones( (coords_and_labels_array_AsH14.shape[0],2))

In [13]:
# Hainault 2 and earls court 2 are 100nm*100nm but also 1024*1024 pixels. Needs to divide the coordinates by 2

coords_and_labels_array_AsH2[:,:2] = (coords_and_labels_array_AsH2[:,:2]//2).astype(np.uint64)
coords_and_labels_array_AsEC2[:,:2] = (coords_and_labels_array_AsEC2[:,:2]//2).astype(np.uint64)

In [14]:

# function to recentre crop on brightest pixel 
def recentre(crop, scan, coordinate, crop_size, min_border, max_border):
        #print(scan.shape, crop_size, crop.shape)
        crop_shape = crop.shape
        # make a copy of the crop that has no pixel below 0
        crop_copy = crop-np.min(crop)
        cropc = np.zeros(crop_shape)
        # use crop only within the specified borders
        cropc[min_border:max_border,min_border:max_border] = crop_copy[min_border:max_border,min_border:max_border].copy()
        # get brightest pixel in the crop
        brightest_pix = np.unravel_index(np.argmax(cropc), cropc.shape)
        # The top-left of the crop is at coordinate - (crop_size - 1) for even or coordinate - crop_size for odd
        # A simpler way is to calculate the offset from the crop center
        crop_center_coord = np.array([d // 2 for d in crop_shape])
        offset = np.array(brightest_pix) - crop_center_coord
        # swap offset[0] and offset[1] to match the coordinate system
        offset = np.array([offset[1], offset[0]])
        new_centre = coordinate.copy() + offset
        # label scan with new centre 
       
        bp = brightest_pix
        if (new_centre != coordinate).any():
            # only redefine the crop if the centre has actually been moved
            x, y = new_centre
            half_size = crop_size // 2
            if crop_size%2 == 0:
                crop = scan[ int(y-half_size):int(y+half_size), int(x-half_size):int(x+half_size)].copy()
                bp = np.unravel_index( np.argmax(crop-np.min(crop)), (crop_size,crop_size) )
            else:
                crop = scan[ int(y-half_size):int(y+half_size+1), int(x-half_size):int(x+half_size+1)].copy()
                bp = np.unravel_index( np.argmax(crop-np.min(crop)), (crop_size,crop_size) )
        return crop, bp, new_centre


In [15]:
# make the crops
def crop_scans(filled_array, empty_array, coords_and_labels_array, crop_size=crop_size, pixel_shift=pixel_shift, res=512):
   # print(1)
    cropped_scans = np.zeros((1,crop_size,crop_size,2))
    coords_labels_copy = coords_and_labels_array.copy()
    new_labels = []
    
    # in case of defects near the edge of the scan, we pad the array by mirroring the pixels along the edges
    filled_mir = np.pad(filled_array, pad_width = 2*pixel_shift, mode='reflect')
    empty_mir = np.pad(empty_array, pad_width = 2*pixel_shift, mode='reflect')
    # because of this, we now need to shift the coordinate of the defects 
    coords_labels_copy[:,:2] += np.array([2*pixel_shift,2*pixel_shift])
   # print(coords_labels_copy[:10,:2])
    # loop through all the labels and take a crop of each feature
    min_border = 5
    max_border = crop_size - min_border
    for i in range(coords_and_labels_array.shape[0]):
        coord = coords_labels_copy[i,:2].copy()
        #print(coord)
        scan_filled = filled_mir[int(coord[1]-pixel_shift):int(coord[1]+pixel_shift), int(coord[0]-pixel_shift):int(coord[0]+pixel_shift)].copy()
        scan_empty = empty_mir[int(coord[1]-pixel_shift):int(coord[1]+pixel_shift), int(coord[0]-pixel_shift):int(coord[0]+pixel_shift)].copy()
        label = coords_labels_copy[i,2]
        # input to the network will be centred on brightest spot (for bright features) so we should do that for training data too
        # we find the brightest pixel separately for the filled and empty state scans so that we can standardise the input to the CNN

        if label == 1 or label == 2 or label == 3 or label == 7:
              # recentre filled states array
              #if label == 7:
              #     fig, ax = plt.subplots(1,2, figsize=(20,10))
              #     ax[0].imshow(scan_filled, cmap='afmhot')
              #     ax[0].set_title('filled state before recentre')
              #     ax[1].imshow(scan_empty, cmap='afmhot')
              #     ax[1].set_title('empty state before recentre')
              #     plt.show() 
              scan_filled, bp1, coord_f = recentre(scan_filled, filled_mir.copy(), coord, crop_size, min_border=min_border, max_border=max_border)
              # recentre empty states array 
              scan_empty, bp2, coord_e = recentre(scan_empty, empty_mir.copy(), coord, crop_size, min_border=min_border, max_border=max_border) 
              #if label == 7:
              #     fig, ax = plt.subplots(1,2, figsize=(20,10))
              #     ax[0].imshow(scan_filled, cmap='afmhot')
              #     ax[0].set_title('filled state after recentre')
              #     ax[1].imshow(scan_empty, cmap='afmhot')
              #     ax[1].set_title('empty state after recentre')
              #     plt.show()
                   
              scan_filled = np.expand_dims(scan_filled, axis=0)
              scan_empty = np.expand_dims(scan_empty, axis=0)
              scan_both = np.stack((scan_filled, scan_empty), axis=-1)
              cropped_scans = np.concatenate((cropped_scans, scan_both), axis=0)
              new_labels.append(label)
        else:
            scan_filled = np.expand_dims(scan_filled, axis=0)
            scan_empty = np.expand_dims(scan_empty, axis=0)
            scan_both = np.stack((scan_filled, scan_empty), axis=-1)
            cropped_scans = np.concatenate((cropped_scans, scan_both), axis=0)
            new_labels.append(label)
            
    labels = np.array(new_labels)-1
    cropped_scans = cropped_scans[1:,:,:,:] # get rid of that initial zero array
    return cropped_scans, labels

In [17]:
cropped_scans_ec, labels_ec = crop_scans(filled_ec, empty_ec, coords_and_labels_array_ec)
cropped_scans_cl, labels_cl = crop_scans(filled_cl, empty_cl, coords_and_labels_array_cl)
cropped_scans_k, labels_k =crop_scans(filled_k, empty_k, coords_and_labels_array_k)
cropped_scans_gr, labels_gr = crop_scans(filled_gr, empty_gr, coords_and_labels_array_gr)
cropped_scans_AsH11, labels_AsH11 =crop_scans(filled_AsH11, empty_AsH11, coords_and_labels_array_AsH11)
cropped_scans_AsH12, labels_AsH12 =crop_scans(filled_AsH12, empty_AsH12, coords_and_labels_array_AsH12)
cropped_scans_AsH13, labels_AsH13 =crop_scans(filled_AsH13, empty_AsH13, coords_and_labels_array_AsH13)
cropped_scans_AsH14, labels_AsH14 =crop_scans(filled_AsH14, empty_AsH14, coords_and_labels_array_AsH14)
cropped_scans_AsH2, labels_AsH2 =crop_scans(filled_AsH2, empty_AsH2, coords_and_labels_array_AsH2)
cropped_scans_AsH0, labels_AsH0 =crop_scans(filled_AsH0, empty_AsH0, coords_and_labels_array_AsH0)

# combine all the crops from the different scans into one numpy array
cropped_scans = np.concatenate((cropped_scans_ec, cropped_scans_gr, cropped_scans_k,
                                cropped_scans_AsH11, cropped_scans_AsH12, cropped_scans_AsH0,
                                cropped_scans_AsH13, cropped_scans_AsH14, cropped_scans_AsH2), axis=0)
labels = np.concatenate((labels_ec, labels_gr,labels_k, labels_AsH11, 
                         labels_AsH12, labels_AsH0, labels_AsH13, labels_AsH14, labels_AsH2), axis=0)

print('Training set:')
for i in range(7): print('label '+str(i)+' has ' + str(np.sum(labels == i)))

# make the validation set

cropped_scans_cl, labels_cl = crop_scans(filled_cl, empty_cl, coords_and_labels_array_cl)
cropped_scans_AsEC2, labels_AsEC2 =crop_scans(filled_AsEC2, empty_AsEC2, coords_and_labels_array_AsEC2)

cropped_scans_val = np.concatenate((cropped_scans_cl, cropped_scans_AsEC2), axis=0)
labels_val = np.concatenate((labels_cl, labels_AsEC2), axis=0)#

print('Validation set:')
for i in range(7): print('label '+str(i)+' has ' + str(np.sum(labels_val == i)))

# make the test set

cropped_scans_AsEC, labels_AsEC =crop_scans(filled_AsEC, empty_AsEC, coords_and_labels_array_AsEC)
cropped_scans_cl2, labels_cl2 = crop_scans(filled_cl2, empty_cl2, coords_and_labels_array_cl2)

cropped_scans_test = np.concatenate((cropped_scans_AsEC, cropped_scans_cl2), axis=0)
labels_test = np.concatenate((labels_AsEC, labels_cl2), axis=0)

print('Test set:')
for i in range(7): print('label '+str(i)+' has ' + str(np.sum(labels_test == i)))


Training set:
label 0 has 251
label 1 has 160
label 2 has 97
label 3 has 227
label 4 has 237
label 5 has 166
label 6 has 349
Validation set:
label 0 has 63
label 1 has 37
label 2 has 27
label 3 has 64
label 4 has 49
label 5 has 93
label 6 has 22
Test set:
label 0 has 35
label 1 has 19
label 2 has 37
label 3 has 22
label 4 has 52
label 5 has 0
label 6 has 71


In [18]:
train_scans = [cropped_scans_ec, cropped_scans_AsH2, cropped_scans_gr, cropped_scans_k, cropped_scans_AsH11, cropped_scans_AsH12, cropped_scans_AsH13, cropped_scans_AsH14, cropped_scans_AsEC2]
train_scan_names = ['cropped_scans_ec', 'cropped_scans_AsH2', 'cropped_scans_gr', 'cropped_scans_k', 'cropped_scans_AsH11', 'cropped_scans_AsH12', 'cropped_scans_AsH13', 'cropped_scans_AsH14', 'cropped_scans_AsEC2']
train_labels = [labels_ec, labels_AsH2, labels_gr, labels_k, labels_AsH11, labels_AsH12, labels_AsH13, labels_AsH14, labels_AsEC2]
val_scans = [cropped_scans_cl, cropped_scans_AsH0]
val_scan_names = ['cropped_scans_cl', 'cropped_scans_AsH0']
val_labels = [labels_cl, labels_AsH0]
test_scans = [cropped_scans_cl2, cropped_scans_AsEC]
test_scan_names = ['cropped_scans_cl2', 'cropped_scans_AsEC']
test_labels = [labels_cl2, labels_AsEC]

In [420]:
# print the number of labels in each image


for j in [0,1,2,6]:
    print('-'*50)
    print('Label ' + str(j) + ' statistics:')
    print('-'*50)
    print('Train set')
    total_train = 0
    for i, scan in enumerate(train_scans):
        #for j in range(7):
            print(train_scan_names[i] + ' label ' + str(j) + ' has ' + str(np.sum(train_labels[i] == j)))
            total_train += np.sum(train_labels[i] == j)
    print('Total number of labels in training set: ' + str(total_train))

    print('-'*50)
    print('-'*50)
    print('Validation set')
    total_val = 0
    for i, scan in enumerate(val_scans):
        #for j in range(7):
            print(val_scan_names[i] + ' label ' + str(j) + ' has ' + str(np.sum(val_labels[i] == j)))   
            total_val += np.sum(val_labels[i] == j)
    print('Total number of labels in validation set: ' + str(total_val))

    print('-'*50)
    print('-'*50)
    print('Test set')
    total_test = 0
    for i, scan in enumerate(test_scans):
        #for j in range(7):
            print(test_scan_names[i] + ' label ' + str(j) + ' has ' + str(np.sum(test_labels[i] == j)))
            total_test += np.sum(test_labels[i] == j)
    print('Total number of labels in test set: ' + str(total_test))

    print('Percentage of labels in training set: ' + str(total_train/(total_train+total_val+total_test)*100) + '%')
    print('Percentage of labels in validation set: ' + str(total_val/(total_train+total_val+total_test)*100) + '%')
    print('Percentage of labels in test set: ' + str(total_test/(total_train+total_val+total_test)*100) + '%')
    print('-'*50)
    print('-'*50)



--------------------------------------------------
Label 0 statistics:
--------------------------------------------------
Train set
cropped_scans_ec label 0 has 9
cropped_scans_AsH2 label 0 has 0
cropped_scans_gr label 0 has 84
cropped_scans_k label 0 has 158
cropped_scans_AsH11 label 0 has 0
cropped_scans_AsH12 label 0 has 0
cropped_scans_AsH13 label 0 has 0
cropped_scans_AsH14 label 0 has 0
cropped_scans_AsEC2 label 0 has 0
Total number of labels in training set: 251
--------------------------------------------------
--------------------------------------------------
Validation set
cropped_scans_cl label 0 has 63
cropped_scans_AsH0 label 0 has 0
Total number of labels in validation set: 63
--------------------------------------------------
--------------------------------------------------
Test set
cropped_scans_cl2 label 0 has 35
cropped_scans_AsEC label 0 has 0
Total number of labels in test set: 35
Percentage of labels in training set: 71.91977077363897%
Percentage of labels in va

In [19]:
cropped_scans.shape, labels.shape, cropped_scans_val.shape, labels_val.shape # check how much data I have


((1487, 14, 14, 2), (1487,), (355, 14, 14, 2), (355,))

Now do some data augmentation. I rotate each data point by 90, 180, and 270 degrees and also take the mirror of each data point and its rotations.

In [20]:
def rot_flip(crops, labels):
    copy_of_crop = np.copy(crops)
    flipped = np.fliplr(copy_of_crop)
    crops = np.concatenate((crops, flipped), axis=0)

    for i in range(3):
        rot = np.rot90(copy_of_crop, k=(i+1), axes=(1,2) )
        crops = np.concatenate((crops, rot), axis=0)
        flipped = np.fliplr(rot)
        crops = np.concatenate((crops, flipped), axis=0)
    
    labels = np.tile(labels, 8) # now need to extend the label array too but everythings in the same order so can just repeat it 8 times.
    #crops=np.transpose(crops, (3,0,1,2))
    return crops, labels


In [21]:
def sigmoid(x, a,b):
    '''
    A sigmoid function that takes in an array x and parameters a and b.
    This is used to rescale the pixel values in the "doubled" image (the brighter ones
    are often the ones that are doubled more clearly).
    args:
    x: array of pixel values
    a: parameter of the sigmoid function
    b: parameter of the sigmoid function
    returns:
    array of pixel values rescaled by the sigmoid function
    '''
    if not isinstance(x, torch.Tensor):
        x = torch.tensor(x, dtype=torch.float32)
    output = 1 / (1 + np.exp(a-b*x))
    output = np.array(output)
    return output

def add_blur_and_noise(crops, labels, do_double_tip = True, min_sigma=0.1, max_sigma=0.5, gauss_kern_size = 5, min_noise_level = 0.1, max_noise_level=0.5, plot=False):
    blurred_noisy_crops = []    

    i = 0
    for crop in crops:
        N = crop_size * 10
        # first upsample to (N,N)
        crop_rs = cv2.resize(crop, (N, N), interpolation=cv2.INTER_LINEAR)
        # choose a random sigma for Gaussian blur
        sigma = np.random.uniform(min_sigma, max_sigma)  
        # choose a random noise level
        noise_level = np.random.uniform(min_noise_level, max_noise_level)
       # print('noise level: ', noise_level)
        # Apply Gaussian blur
        blurred_crop = cv2.GaussianBlur(crop_rs, (gauss_kern_size, gauss_kern_size), sigma)
        # Add Gaussian noise
        noise = np.random.normal(0, noise_level, crop_rs.shape)
        blurred_noisy_crop = blurred_crop + noise 
        # now add a slight double tip with 50% probability
        if do_double_tip:
            double_tip = np.random.rand() < 0.5
            if double_tip:
                b_min, b_max = 8, 10
                a_min, a_max = 7, 9
                multiple_min, multiple_max = 1, 2.5
                # get random values for a and b
                b = np.random.uniform(b_min, b_max)
                a = np.random.uniform(a_min, a_max)
                multiple = np.random.uniform(multiple_min, multiple_max)        
                # Apply the sigmoid function to each pixel value
                sigmoid_array = multiple*sigmoid(blurred_noisy_crop,a,b)
                # cap sigmoid_array to 1
                sigmoid_array = (sigmoid_array - np.min(sigmoid_array))/(np.max(sigmoid_array)-np.min(sigmoid_array))
                # Offset each pixel by a random value in x and y (between 5 and 10)
                # Generate random offsets for x and y
                offset_x = torch.randint(5, 15, (1,))[0] 
                offset_y = torch.randint(5, 15, (1,))[0]
                # make negative with 50% chance
                if random.random() < 0.5:
                    offset_x = -offset_x
                if random.random() < 0.5:
                    offset_y = -offset_y
                # apply a random nxn kernel
                # n is random integer between 2 and 8
                n = np.random.randint(30, 50)
                kernel = np.random.uniform(-0.5, 1, (1,n,n))
                kernel = kernel/np.sum(kernel)
                #print('kernel size: ', kernel.shape)
                sigmoid_array_filled = torch.tensor(sigmoid_array[:,:,0], dtype=torch.float64).unsqueeze(0)
                sigmoid_array_empty = torch.tensor(sigmoid_array[:,:,1], dtype=torch.float64).unsqueeze(0)
                filtered_offset_filled = torch.nn.functional.conv2d(sigmoid_array_filled, torch.tensor(kernel).unsqueeze(0), padding='same')
                filtered_offset_filled = filtered_offset_filled.squeeze(0)
                filtered_offset_filled = filtered_offset_filled.numpy() 
                filtered_offset_empty = torch.nn.functional.conv2d(sigmoid_array_empty, torch.tensor(kernel).unsqueeze(0), padding='same')
                filtered_offset_empty = filtered_offset_empty.squeeze(0)
                filtered_offset_empty = filtered_offset_empty.numpy()
                # stack the two arrays together
                filtered_offset = np.stack((filtered_offset_filled, filtered_offset_empty), axis=-1)       
                # Add the add_array to the base_array with the offset
                # then crop it so it only includes the parts where the two arrays are overlaid
                # Create a copy of the base array
                result = np.copy(blurred_noisy_crop)
                rows, cols,_ = blurred_noisy_crop.shape
                if offset_x >= 0 and offset_y >= 0:
                    # Positive offsets
                    result[offset_x:, offset_y:,:] += filtered_offset[:rows-offset_x, :cols-offset_y,:]
                elif offset_x >= 0 and offset_y < 0:
                    # Negative y-offset
                    result[offset_x:, :offset_y,:] += filtered_offset[:rows-offset_x, -offset_y:,:]
                elif offset_x < 0 and offset_y >= 0:
                    # Negative x-offset
                    result[:offset_x, offset_y:,:] += filtered_offset[-offset_x:, :cols-offset_y,:]
                else:  # offset_x < 0 and offset_y < 0
                    # Negative offsets for both x and y
                    result[:offset_x, :offset_y,:] += filtered_offset[-offset_x:, -offset_y:,:]
            
            # downsample back to (crop_size,crop_size)
            if double_tip:
                blurred_noisy_crop_rs = cv2.resize(result, (crop_size, crop_size), interpolation=cv2.INTER_LINEAR)
            else:    
                blurred_noisy_crop_rs = cv2.resize(blurred_noisy_crop, (crop_size, crop_size), interpolation=cv2.INTER_LINEAR)
        
        else:    
            blurred_noisy_crop_rs = cv2.resize(blurred_noisy_crop, (crop_size, crop_size), interpolation=cv2.INTER_LINEAR)
            
        blurred_noisy_crop_rs = blurred_noisy_crop_rs[1:crop_size-1, 1:crop_size-1]
    
        # Append to the list  
        if plot:
            if 200 < i and i < 220:
                print('Double tip: ', double_tip)    
                print('offset_x: ', offset_x, ' offset_y: ', offset_y)
                print('n: ', n)
                label = labels[i]
                if label == 6 or label == 0 or label == 1 or label == 2:
                    print('sigma: ', sigma) 
                    print('noise level: ', noise_level)
                    # plot everything together
                    fig, axs = plt.subplots(2, 3, figsize=(10, 10))
                    axs[0,0].imshow(crop[1:crop_size-1,1:crop_size-1,0], cmap='afmhot')
                    axs[0,0].set_title('Original Crop')
                    axs[0,1].imshow(crop_rs[:,:,0], cmap='afmhot')
                    axs[0,1].set_title('Upsampled Crop')
                    axs[0,2].imshow(filtered_offset_filled, cmap='afmhot')
                    axs[0,2].set_title('Filtered Offset Filled')
                    axs[1,0].imshow(blurred_noisy_crop[:,:,0], cmap='afmhot')
                    axs[1,0].set_title('Blurred and noisy Crop')
                    axs[1,1].imshow(blurred_noisy_crop_rs[:,:,0], cmap='afmhot')
                    axs[1,1].set_title('Blurred and Noisy Crop downsampled')
                    axs[1,2].imshow(crop[1:crop_size-1,1:crop_size-1,0] - blurred_noisy_crop_rs[:,:,0], cmap='afmhot')
                    axs[1,2].set_title('Difference')
                    plt.show()
                # print the old and new means + variances
                print('Old mean: ', np.mean(crop[1:crop_size-1,1:crop_size-1,0]), ' Old variance: ', np.var(crop[1:crop_size-1,1:crop_size-1,0]))
                print('New mean: ', np.mean(blurred_noisy_crop_rs[:,:,0]), ' New variance: ', np.var(blurred_noisy_crop_rs[:,:,0]))
        blurred_noisy_crops.append(blurred_noisy_crop_rs)
        if len(blurred_noisy_crops) % 1000 == 0:
            print(f'Processed {len(blurred_noisy_crops)} crops with blur and noise.')
        i += 1

    blurred_noisy_crops = np.array(blurred_noisy_crops)

    # Combine original, blurred, and noisy crops
    combined_crops = np.concatenate((crops[:,1:crop_size-1,1:crop_size-1,:], blurred_noisy_crops), axis=0)
    combined_labels = np.tile(labels, 2)  # Repeat labels for the new crops
    
    return combined_crops, combined_labels

In [22]:
cropped_scans.shape

(1487, 14, 14, 2)

In [23]:
train_data, train_labels = rot_flip(cropped_scans, labels)

In [24]:
train_data, train_labels = add_blur_and_noise(train_data, train_labels, min_sigma=10, max_sigma=100, gauss_kern_size = 21, min_noise_level=0.001, max_noise_level=0.003,plot=False)

C:\Users\nkolev\AppData\Local\Temp\ipykernel_13408\2220995162.py:69: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\Convolution.cpp:1032.)
  filtered_offset_filled = torch.nn.functional.conv2d(sigmoid_array_filled, torch.tensor(kernel).unsqueeze(0), padding='same')


Processed 1000 crops with blur and noise.
Processed 2000 crops with blur and noise.
Processed 3000 crops with blur and noise.
Processed 4000 crops with blur and noise.
Processed 5000 crops with blur and noise.
Processed 6000 crops with blur and noise.
Processed 7000 crops with blur and noise.
Processed 8000 crops with blur and noise.
Processed 9000 crops with blur and noise.
Processed 10000 crops with blur and noise.
Processed 11000 crops with blur and noise.


In [25]:
cropped_scans_val, labels_val = rot_flip(cropped_scans_val, labels_val)
cropped_scans_test, labels_test = rot_flip(cropped_scans_test, labels_test)

In [26]:
cropped_scans_val, labels_val = add_blur_and_noise(cropped_scans_val, labels_val, do_double_tip=False)

Processed 1000 crops with blur and noise.
Processed 2000 crops with blur and noise.


In [1110]:
#cropped_scans_, labels_ = add_blur_and_noise(cropped_scans, labels, min_sigma=10, max_sigma=100, gauss_kern_size = 31, min_noise_level=0.003, max_noise_level=0.005)

In [27]:
cropped_scans_test = cropped_scans_test[:,1:crop_size-1,1:crop_size-1,:]

In [28]:
cropped_scans_val.shape

(5680, 12, 12, 2)

In [29]:
print('Training set after augmentation:')
for i in range(7): print('label '+str(i)+' has ' + str(np.sum(labels == i)))
print('Validation set after augmentation:')
for i in range(7): print('label '+str(i)+' has ' + str(np.sum(labels_val == i)))
print('Test set after augmentation:')
for i in range(7): print('label '+str(i)+' has ' + str(np.sum(labels_test == i)))

Training set after augmentation:
label 0 has 251
label 1 has 160
label 2 has 97
label 3 has 227
label 4 has 237
label 5 has 166
label 6 has 349
Validation set after augmentation:
label 0 has 1008
label 1 has 592
label 2 has 432
label 3 has 1024
label 4 has 784
label 5 has 1488
label 6 has 352
Test set after augmentation:
label 0 has 280
label 1 has 152
label 2 has 296
label 3 has 176
label 4 has 416
label 5 has 0
label 6 has 568


In [30]:
labels.shape, cropped_scans.shape, labels_val.shape, cropped_scans_val.shape

((1487,), (1487, 14, 14, 2), (5680,), (5680, 12, 12, 2))

In [31]:
# transpose so shape is (number of data points, num of channels, res,res)
#cropped_scans = np.transpose(cropped_scans, axes=[0,3,1,2])
train_data = np.transpose(train_data, axes=[0,3,1,2])
cropped_scans_val = np.transpose(cropped_scans_val, axes=[0,3,1,2])
cropped_scans_test = np.transpose(cropped_scans_test, axes=[0,3,1,2])

In [32]:
train_data.shape

(23792, 2, 12, 12)

In [33]:
cropped_scans_val.shape

(5680, 2, 12, 12)

In [ ]:
np.save('DB_train.npy', cropped_scans)
np.save('DB_train_labels.npy', labels)

In [ ]:
np.save('DB_val.npy', cropped_scans_val)
np.save('DB_val_labels.npy', labels_val)

In [ ]:
np.save('DB_test.npy', cropped_scans_test)
np.save('DB_test_labels.npy', labels_test)

In [ ]:
#cropped_scans = np.load('As_train.npy')
#labels = np.load('As_train_labels.npy')


In [462]:
cropped_scans.shape

(23792, 2, 11, 11)

In [463]:
np.sum(labels==6)

5584

In [34]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [35]:
# data set class
class CustomDataset(Dataset):
    def __init__(self, images, labels, features,length=None):
        self.images = images
        self.labels = labels
        self.features = features
        self.refine()
        self.length = length if length is not None else len(self.images)


    def refine(self):
        # gets rid of unwanted features in the dataset and fixes labels correspondingly
        crops = [self.images[self.labels==i,:,:,:] for i in self.features]
        self.images = torch.vstack(crops)
        
        labels = [self.labels[self.labels==i] for i in self.features]
        new_labels = []
        
        for i, array in enumerate(labels):
            new_labels.append(i*np.ones(array.shape))
        self.labels = torch.tensor(np.hstack(new_labels)).long()
        print('Dataset refined. Number of images: ', len(self.images), ' Number of labels: ', len(self.labels))
        
        return 

    def __len__(self):
        return self.length 

    def __getitem__(self, idx):
        idx = idx % self.length  # Ensure idx is within bounds
        label = torch.clone(self.labels[idx])
        image = torch.clone(self.images[idx])
       # print('before: ', torch.mean(image[0,:,:]), torch.var(image[0,:,:]) , torch.mean(image[1,:,:]))
        image[0,:,:] = (image[0,:,:]-torch.mean(image[0,:,:]))#/torch.std(image[0,:,:])
        image[1,:,:] = (image[1,:,:]-torch.mean(image[1,:,:]))#/torch.std(image[1,:,:])
       # print('after: ', torch.mean(image[0,:,:]), torch.var(image[0,:,:]) , torch.mean(image[1,:,:]))
        return image, label

In [36]:
cropped_scans_val.shape

(5680, 2, 12, 12)

In [38]:
train_dataset = CustomDataset(torch.tensor(np.copy(train_data)).float(), torch.tensor(train_labels).long(), [0,1,2])
val_data = CustomDataset(torch.tensor(np.copy(cropped_scans_val)).float(), torch.tensor(labels_val).long(), [0,1,2])
test_data = CustomDataset(torch.tensor(np.copy(cropped_scans_test)).float(), torch.tensor(labels_test).long(), [0,1,2])

Dataset refined. Number of images:  8128  Number of labels:  8128
Dataset refined. Number of images:  2032  Number of labels:  2032
Dataset refined. Number of images:  728  Number of labels:  728


In [39]:
# Number of examples to show per label
n_examples = 5

print("Plotting examples from train_data")

# The labels in train_data are 0, 1, 2, 3, corresponding to original labels 0, 1, 2, 6
for label_idx in range(4):
    print(f"--- Showing examples for new label: {label_idx} ---")
    
    # Find the indices of images with the current label
    indices = np.where(train_data.labels.numpy() == label_idx)[0]
    
    # Check if there are any images with this label
    if len(indices) == 0:
        print(f"No examples found for label {label_idx}")
        continue

    # Take the first n_examples, or fewer if not enough are available
    num_to_show = min(n_examples, len(indices))
    
    for i in range(num_to_show):
        # Get an image and its label from the dataset
        image, label = train_data[indices[i+500]]
        
        # Create a figure with two subplots side-by-side
        fig, ax = plt.subplots(1, 2, figsize=(8, 4))
        
        # Display the first channel (filled state)
        ax[0].imshow(image[0], cmap='afmhot')
        ax[0].set_title('Filled State')
        ax[0].axis('off')

        # Display the second channel (empty state)
        ax[1].imshow(image[1], cmap='afmhot')
        ax[1].set_title('Empty State')
        ax[1].axis('off')

        # Set a title for the entire figure
        plt.suptitle(f'Label: {label.item()}, Example: {i+1}')
        plt.show()

Plotting examples from train_data
--- Showing examples for new label: 0 ---


AttributeError: 'numpy.ndarray' object has no attribute 'labels'

In [76]:
print('Train data labels after dataset creation:')
for i in range(7): print('label '+str(i)+' has ' + str(len(np.where(train_dataset.labels == i)[0])))
print('Validation data labels after dataset creation:')
for i in range(7): print('label '+str(i)+' has ' + str(len(np.where(val_data.labels == i)[0])))
print('Test data labels after dataset creation:')
for i in range(7): print('label '+str(i)+' has ' + str(len(np.where(test_data.labels == i)[0])))


Train data labels after dataset creation:
label 0 has 4016
label 1 has 2560
label 2 has 1552
label 3 has 0
label 4 has 0
label 5 has 0
label 6 has 0
Validation data labels after dataset creation:
label 0 has 1008
label 1 has 592
label 2 has 432
label 3 has 0
label 4 has 0
label 5 has 0
label 6 has 0
Test data labels after dataset creation:
label 0 has 280
label 1 has 152
label 2 has 296
label 3 has 0
label 4 has 0
label 5 has 0
label 6 has 0


In [1090]:
cropped_scans.shape

(23792, 2, 20, 20)

In [52]:
# Compute the sample weights
class_sample_count = np.array([len(np.where(train_dataset.labels== t)[0]) for t in range(3)])
print(class_sample_count)
weight = 1.0 / class_sample_count
samples_weight = np.array(weight)/np.sum(weight)

batch_size = 50

# Create the sampler and data loader
data_loader_train = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

data_loader_val = DataLoader(val_data, batch_size=batch_size, shuffle=True)

data_loader_test = DataLoader(test_data, batch_size=batch_size, shuffle=True)
#ec_data_loader = DataLoader(ec_data, batch_size=batch_size, shuffle=True)

[4016 2560 1552]


In [53]:
def calculate_mean_variance(dataloader, n):
    samples_loaded = 0
    channel_means = []
    channel_variances = []

    for crops, _ in dataloader:
        for crop in crops:
            if samples_loaded >= n:
                break
            channel_means.append(torch.mean(crop, dim=(1, 2)))
            channel_variances.append(torch.var(crop, dim=(1, 2)))
            samples_loaded += 1
        if samples_loaded >= n:
            break
    print(channel_means)
    print(channel_variances)
    channel_means = torch.stack(channel_means).mean(dim=0)
    channel_variances = torch.stack(channel_variances).mean(dim=0)
    print(channel_means)
    print("Channel-wise Mean:", channel_means)
    print("Channel-wise Variance:", channel_variances)
    return channel_means, channel_variances

# Example usage
n_samples = 10
channel_means, channel_variances = calculate_mean_variance(data_loader_train, n_samples)

[tensor([-1.4487e-08,  1.6350e-08]), tensor([-2.3180e-08, -4.1806e-08]), tensor([-2.4318e-08,  8.3819e-09]), tensor([-8.0715e-09, -1.0141e-08]), tensor([ 4.1392e-09, -4.1392e-10]), tensor([3.5183e-09, 3.8288e-08]), tensor([-1.5936e-08, -4.9671e-09]), tensor([6.9332e-09, 1.7126e-08]), tensor([ 1.8109e-08, -7.5541e-09]), tensor([2.1938e-08, 2.5663e-08])]
[tensor([0.0177, 0.0160]), tensor([0.0171, 0.0665]), tensor([0.0030, 0.0031]), tensor([0.0006, 0.0012]), tensor([0.0007, 0.0045]), tensor([0.0142, 0.0678]), tensor([0.0016, 0.0055]), tensor([0.0028, 0.0027]), tensor([0.0622, 0.0208]), tensor([0.0015, 0.0011])]
tensor([-3.1355e-09,  4.0926e-09])
Channel-wise Mean: tensor([-3.1355e-09,  4.0926e-09])
Channel-wise Variance: tensor([0.0121, 0.0189])


In [54]:
class_sample_count, samples_weight

(array([4016, 2560, 1552]), array([0.19393455, 0.30423482, 0.50183063]))

In [55]:
samples_weight

array([0.19393455, 0.30423482, 0.50183063])

In [45]:
class NeuralNetwork(nn.Module):
    def __init__(self, channels, crop_size, n_outputs, fc_layers, fc_nodes, dropout):
        super().__init__()
        self.fc_layers = fc_layers
        self.convolutional_relu_stack = nn.Sequential(
            nn.Conv2d(channels, 32, 3, stride=1, padding='valid'),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, stride=1, padding='valid'),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 128, 3, stride=1, padding='valid'),
            nn.BatchNorm2d(128),
            nn.ReLU(),            
            nn.Conv2d(128, 256, 3, stride=1, padding='valid'),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Flatten(),
            nn.LazyLinear(out_features=fc_nodes),
            nn.Dropout(dropout),
            nn.ReLU(),
            nn.BatchNorm1d(fc_nodes),
        )
        
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(fc_nodes, fc_nodes),
            nn.Dropout(dropout),
            nn.ReLU(),
            nn.BatchNorm1d(fc_nodes)
        )
        
        self.linear_relu_stack_last = nn.Sequential(
            nn.Linear(fc_nodes, n_outputs)
        )
        

    def forward(self, x, training = True):
        x = self.convolutional_relu_stack(x)
        for i in range(self.fc_layers-1):
            x = self.linear_relu_stack(x)
        logits = self.linear_relu_stack_last(x) 
        if training == True:
            return logits
        else: 
            logits = torch.nn.functional.softmax(logits, dim=1)
            return logits

In [768]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [734]:
model = NeuralNetwork(channels=2, crop_size=11, n_outputs=4, fc_layers=2, fc_nodes=100, dropout=0.2).to(device)

In [60]:
# Function to save the model
def save_model(model, path):
    torch.save(model.state_dict(), path)

In [ ]:
# test accuracy function
def testAccuracy(model, dataloader, device='cuda'):

    model.eval()
    #accuracy = 0.0
    #total = 0.0
    predictions = []
    all_labels = [] 
    with torch.no_grad():
        for data in dataloader:
            crops, labels = data
           # total += labels.size(0)
            crops, labels = crops.to(device), labels.to(device)
            # run the model on the test set to predict labels
            outputs = model(crops.float())
            
            # the label with the highest prob will be our prediction
            predicted = torch.argmax(outputs.data, 1)
            #print(predicted)
            predictions.append(predicted.unsqueeze(1))
            all_labels.append(labels.unsqueeze(1))

            #accuracy += (predicted == labels).sum().item()
    # stack predictions and labels into 1 array each          
    predictions = torch.vstack(predictions)
    all_labels = torch.vstack(all_labels)
    conf = confusion_matrix(all_labels.cpu(), predictions.cpu())
    
    print(conf)    

    accuracy = 100 * np.sum(np.diag(conf)) / np.sum(conf)
        
    return accuracy

In [ ]:
def train(model, dataloader_train, dataloader_test, loss_, num_epochs, path, optimizer, patience = 25, scheduler=None, device='cuda'):
   
    best_loss = np.inf
    best_accuracy = 0
    patience_counter = 0 
    # Iterate over the training data
    for epoch in range(num_epochs):
        running_train_loss = 0.0
        running_test_loss = 0.0

        # train the model
        model.train()
        for i, (crops, labels) in enumerate(dataloader_train):
            # Get the crops and labels
            crops, labels = crops.to(device), labels.to(device)
           
            # Zero the parameter gradients
            optimizer.zero_grad()
            # get prediction
            outputs = model(crops.float()).float().to(device)
            
            loss = loss_(outputs, labels.long())
            running_train_loss += loss.item()
        
            # Backward pass
            loss.backward()
            optimizer.step()
            
        if scheduler is not None:
            scheduler.step()
        
        print('Confusion matrix from train data')
        accuracy = testAccuracy(model, dataloader_train, device = device)
        print('epoch', epoch, 'train accuracy over whole train set: %d %%' % (accuracy))
            
        # get the test accuracy
        model.eval()
        for i, (crops, labels) in enumerate(dataloader_test):
            # Get the crops and labels
            crops, labels = crops.to(device), labels.to(device)
            # get prediction and loss
            pred = model(crops.float())
            loss = loss_(pred, labels.long())
            
            running_test_loss += loss.item()
            #
        print('Confusion matrix from va.idation data')
        accuracy = testAccuracy(model, dataloader_test, device=device)
        print('epoch', epoch, 'test accuracy over whole validation set: %d %%' % (accuracy))

        # save the model if the loss is the best
        if (running_test_loss/len(dataloader_test)) < best_loss:
            print('Saving model from epoch', epoch)
            save_model(model, path)
            best_loss = running_test_loss/len(dataloader_test)
            patience_counter = 0
            best_accuracy = accuracy
        elif (running_test_loss/len(dataloader_test)) == best_loss:
            if accuracy > best_accuracy:
                print('Saving model from epoch', epoch)
                save_model(model, path)
                best_accuracy = accuracy
                patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print('Early stopping at epoch', epoch)
                break

    
        print('Epoch: %d loss: %.3f' % (epoch , running_test_loss / len(dataloader_test)))

In [892]:
class FocalLoss(nn.Module):
    def __init__(self, gamma=2, alpha=None, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = nn.functional.cross_entropy(inputs, targets, reduction='none', weight=self.alpha)
        pt = torch.exp(-ce_loss)  # = softmax prob for the true class
        focal_loss = ((1 - pt) ** self.gamma) * ce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

In [40]:
# find percentage of each label in the training set
def find_percentage():
    percentages_train = {}
    percentages_val = {}
    percentages_test = {}
    for i in range(4):
        total = torch.sum(train_data.labels==i) + torch.sum(val_data.labels==i) + torch.sum(test_data.labels==i)
        percentages_train[i] = torch.sum(train_data.labels==i) / total * 100
        percentages_val[i] = torch.sum(val_data.labels==i) / total * 100
        percentages_test[i] = torch.sum(test_data.labels==i) / total * 100
    
    print('Percentage of each label in the training set:')
    for i in range(4):
        print('label ' + str(i) + ': ' + str(percentages_train[i].item()) + '%')
    print('Percentage of each label in the validation set:')
    for i in range(4):
        print('label ' + str(i) + ': ' + str(percentages_val[i].item()) + '%')
    print('Percentage of each label in the test set:')
    for i in range(4):
        print('label ' + str(i) + ': ' + str(percentages_test[i].item()) + '%')

find_percentage()


AttributeError: 'numpy.ndarray' object has no attribute 'labels'

In [92]:
def train_and_test(model, path, dataloader_train, dataloader_val, test_data, samples_weight, device='cuda', num_epochs=100):

    # Define the loss functions, and optimizer
    #loss = FocalLoss(gamma=2, alpha=torch.tensor(samples_weight).float()) # focal loss function
    loss = nn.CrossEntropyLoss(weight=torch.tensor(samples_weight).float()) # cross entropy loss function
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 10, gamma=0.01) # lr=0.1*lr after every 5 epochs

    # train the model
    train(model, dataloader_train, dataloader_val, loss, num_epochs, path, optimizer, patience=10, scheduler=scheduler,device=device)
    
    # load best model
    model.load_state_dict(torch.load(path))
    
    # test the model
    t_data = torch.clone(test_data.images)
    for image in t_data:
        image[0,:,:] = image[0,:,:]-torch.mean(image[0,:,:])
        image[1,:,:] = image[1,:,:]-torch.mean(image[1,:,:])

    y_pred1 = model(t_data)
    conf = confusion_matrix(test_data.labels.cpu(), torch.argmax(y_pred1.cpu(), axis=1) )
    
    print("__"*50)
    print("Stats for ", path)

    print(conf)    

    num_classes = conf.shape[0]
    precision = np.zeros(num_classes)
    recall = np.zeros(num_classes)

    for i in range(num_classes):
        tp = conf[i, i]
        fp = np.sum(conf[:, i]) - tp
        fn = np.sum(conf[i, :]) - tp
        
        precision[i] = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall[i] = tp / (tp + fn) if (tp + fn) > 0 else 0
        
        print(f"Class {i}:")
        print(f"  Precision: {precision[i]:.4f}")
        print(f"  Recall:    {recall[i]:.4f}")

        # Micro-averaged Precision and Recall
        total_tp = np.sum(np.diag(conf))
        total_fp = np.sum(conf) - total_tp # Sum of all FPs is sum of all elements minus sum of diagonal
        total_fn = np.sum(conf) - total_tp # Sum of all FNs is sum of all elements minus sum of diagonal

        micro_precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
        micro_recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
        # Note: For multi-class, micro-precision = micro-recall = micro-F1 = accuracy
        print(f"Micro-averaged Precision: {micro_precision:.4f}")
        print(f"Micro-averaged Recall:    {micro_recall:.4f}")


    # Overall accuracy (already available from np.sum(np.diag(conf)) / np.sum(conf))
    # but can be recalculated for verification
    accuracy = np.sum(np.diag(conf)) / np.sum(conf)
    print(f"\nOverall Accuracy: {accuracy:.4f}")

    # Macro-averaged Precision and Recall
    macro_precision = np.mean(precision)
    macro_recall = np.mean(recall)
    print(f"Macro-averaged Precision: {macro_precision:.4f}")
    print(f"Macro-averaged Recall:    {macro_recall:.4f}")

    print("__"*50)

    return model, conf, precision, recall, accuracy, macro_precision, macro_recall, micro_precision, micro_recall

In [59]:
def load_model(model, file_path):
    model.load_state_dict(torch.load(file_path, map_location=torch.device('cpu') ) )
    return model

In [110]:
# lists of model parameters to try
# model = NeuralNetwork(channels=2, crop_size=11, n_outputs=4, fc_layers=2, fc_nodes=200, dropout=0.2).to(device)
fc_layers = [2]
fc_nodes = [256,512]
dropouts = [0.2]

In [111]:
# dictionary to store the results
models_results = {}

for fc_layer in fc_layers:
    for fc_node in fc_nodes:
        for dropout in dropouts:
            model = NeuralNetwork(channels=2, crop_size=crop_size-2, n_outputs=3, fc_layers=fc_layer, fc_nodes=fc_node, dropout=dropout).to(device)
            path = f"model_fc{fc_layer}_nodes{fc_node}_drop{dropout}_CEloss.pth"
            print(f"Training model with {fc_layer} FC layers, {fc_node} nodes, and {dropout} dropout")
            model.to(device)
            model, conf, precision, recall, accuracy, macro_precision, macro_recall, micro_precision, micro_recall = train_and_test(model, path, data_loader_train, data_loader_val, test_data,samples_weight,device=device)
            models_results[(fc_layer, fc_node, dropout)] = {
                "model": model,
                "confusion_matrix": conf,
                "precision": precision,
                "recall": recall,
                "accuracy": accuracy,
                "macro_precision": macro_precision,
                "macro_recall": macro_recall,
                "micro_precision": micro_precision,
                "micro_recall": micro_recall
            }
    

Training model with 2 FC layers, 256 nodes, and 0.2 dropout
Confusion matrix from train data
[[3330  127  559]
 [ 630 1698  232]
 [ 549  281  722]]
epoch 0 train accuracy over whole train set: 70 %
Confusion matrix from test data
[[658   5 345]
 [226 157 209]
 [137  38 257]]
epoch 0 test accuracy over whole validation set: 52 %
Saving model from epoch 0
Epoch: 0 loss: 1.142
Confusion matrix from train data
[[3587   23  406]
 [ 237 2000  323]
 [ 223  148 1181]]
epoch 1 train accuracy over whole train set: 83 %
Confusion matrix from test data
[[354  12 642]
 [ 31 150 411]
 [  1  28 403]]
epoch 1 test accuracy over whole validation set: 44 %
Epoch: 1 loss: 1.507
Confusion matrix from train data
[[3336  365  315]
 [ 123 2278  159]
 [ 201  396  955]]
epoch 2 train accuracy over whole train set: 80 %
Confusion matrix from test data
[[301  29 678]
 [  0 265 327]
 [ 10  32 390]]
epoch 2 test accuracy over whole validation set: 47 %
Epoch: 2 loss: 1.147
Confusion matrix from train data
[[3385  

C:\Users\nkolev\AppData\Local\Temp\ipykernel_13408\1497900860.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path))


____________________________________________________________________________________________________
Stats for  model_fc2_nodes256_drop0.2_CEloss.pth
[[262  18   0]
 [ 87  65   0]
 [160   7 129]]
Class 0:
  Precision: 0.5147
  Recall:    0.9357
Micro-averaged Precision: 0.6264
Micro-averaged Recall:    0.6264
Class 1:
  Precision: 0.7222
  Recall:    0.4276
Micro-averaged Precision: 0.6264
Micro-averaged Recall:    0.6264
Class 2:
  Precision: 1.0000
  Recall:    0.4358
Micro-averaged Precision: 0.6264
Micro-averaged Recall:    0.6264

Overall Accuracy: 0.6264
Macro-averaged Precision: 0.7457
Macro-averaged Recall:    0.5997
____________________________________________________________________________________________________
Training model with 2 FC layers, 512 nodes, and 0.2 dropout
Confusion matrix from train data
[[3151  353  512]
 [ 375 1913  272]
 [ 335  323  894]]
epoch 0 train accuracy over whole train set: 73 %
Confusion matrix from test data
[[290  48 670]
 [ 54 290 248]
 [ 34 

C:\Users\nkolev\AppData\Local\Temp\ipykernel_13408\1497900860.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path))


____________________________________________________________________________________________________
Stats for  model_fc2_nodes512_drop0.2_CEloss.pth
[[275   0   5]
 [  0 102  50]
 [117  12 167]]
Class 0:
  Precision: 0.7015
  Recall:    0.9821
Micro-averaged Precision: 0.7473
Micro-averaged Recall:    0.7473
Class 1:
  Precision: 0.8947
  Recall:    0.6711
Micro-averaged Precision: 0.7473
Micro-averaged Recall:    0.7473
Class 2:
  Precision: 0.7523
  Recall:    0.5642
Micro-averaged Precision: 0.7473
Micro-averaged Recall:    0.7473

Overall Accuracy: 0.7473
Macro-averaged Precision: 0.7828
Macro-averaged Recall:    0.7391
____________________________________________________________________________________________________


In [ ]:
# put all results into a pandas DataFrame for easy viewing

results_df = pd.DataFrame.from_dict(
    {k: v for k, v in models_results.items()},
   orient='index'
)

results_df = results_df.reset_index()
results_df.columns = [
    'fc_layers', 'fc_nodes', 'dropout', 'model', 'confusion_matrix', 
    'precision', 'recall', 'accuracy', 'macro_precision', 
    'macro_recall', 'micro_precision', 'micro_recall'
]

results_df

,fc_layers,fc_nodes,dropout,model,confusion_matrix,precision,recall,accuracy,macro_precision,macro_recall,micro_precision,micro_recall
0,2,1024,0.2,NeuralNetwork(\n (convolutional_relu_stack): ...,"[[264, 5, 3, 8], [0, 145, 7, 0], [68, 30, 144,...","[0.7764705882352941, 0.7880434782608695, 0.832...","[0.9428571428571428, 0.9539473684210527, 0.486...",0.841049,0.823345,0.832178,0.841049,0.841049


In [1335]:
results_df = results_df.sort_values(by='micro_precision', ascending=False)
results_df

,fc_layers,fc_nodes,dropout,model,confusion_matrix,precision,recall,accuracy,macro_precision,macro_recall,micro_precision,micro_recall
0,2,512,0.2,NeuralNetwork(\n (convolutional_relu_stack): ...,"[[267, 1, 11, 1], [0, 148, 4, 0], [24, 27, 217...","[0.898989898989899, 0.8176795580110497, 0.7587...","[0.9535714285714286, 0.9736842105263158, 0.733...",0.875772,0.855225,0.886482,0.875772,0.875772
1,2,256,0.2,NeuralNetwork(\n (convolutional_relu_stack): ...,"[[271, 0, 9, 0], [0, 130, 22, 0], [25, 14, 239...","[0.8914473684210527, 0.896551724137931, 0.6424...","[0.9678571428571429, 0.8552631578947368, 0.807...",0.846451,0.848144,0.858783,0.846451,0.846451
2,2,128,0.2,NeuralNetwork(\n (convolutional_relu_stack): ...,"[[272, 0, 8, 0], [0, 101, 51, 0], [22, 7, 220,...","[0.9066666666666666, 0.9351851851851852, 0.617...","[0.9714285714285714, 0.6644736842105263, 0.743...",0.831790,0.842871,0.808255,0.831790,0.831790


In [63]:
from sklearn.metrics import confusion_matrix